<a href="https://colab.research.google.com/github/AXB2024/RAG-Pipline-Project/blob/main/Project_5_Final_Task_Measure_your_RAG_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate sentence-transformers faiss-cpu llama-cpp-python unstructured PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 MB 21.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 66.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/

In [ ]:
import os
import fitz  # PyMuPDF
import time
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama


In [ ]:
# STEP 1: Mount / Create Document Folder
os.makedirs("/content/documents", exist_ok=True)

In [ ]:
# STEP 2: Extract Text from PDFs
def extract_text_from_pdfs(folder="/content/documents"):
    docs = {}
    for fname in os.listdir(folder):
        if fname.endswith(".pdf"):
            with fitz.open(os.path.join(folder, fname)) as doc:
                full_text = ""
                for page in doc:
                    full_text += page.get_text()
                docs[fname] = full_text
    return docs

In [ ]:
# STEP 3: RAG Components
queries = {
    "appraisal.pdf": "What is the estimated home value?",
    "sample_bank_statement.pdf": "How much was the last transaction?",
    "payslip_sample_image.pdf": "What is the total net salary for this month?"
}


In [ ]:
def embed_documents(docs, embedder):
    passages = []
    doc_map = []
    for name, text in docs.items():
        for i in range(0, len(text), 300):
            chunk = text[i:i+300]
            passages.append(chunk)
            doc_map.append(name)
    embeddings = embedder.encode(passages, convert_to_tensor=True).cpu().numpy()
    return passages, doc_map, embeddings

In [ ]:
import numpy as np

def search(query, embedder, passages, embeddings):
    query_vec = embedder.encode([query])[0]
    query_vec = np.array(query_vec).astype('float32').reshape(1, -1)

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    D, I = index.search(query_vec, 1)
    return passages[I[0][0]]


In [ ]:
def load_model(name, model_type):
    if model_type == "transformers":
        tokenizer = AutoTokenizer.from_pretrained(name)
        model = AutoModelForCausalLM.from_pretrained(name, device_map="auto", torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
        pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
        return lambda prompt: pipe(prompt, max_new_tokens=128, do_sample=True)[0]['generated_text']
    elif model_type == "llama-cpp":
        return Llama(model_path=name, n_ctx=2048, n_threads=4)

In [ ]:
def generate_answer(model, query, context, model_type):
    prompt = f"Answer this question based on the context:\nContext: {context}\nQuestion: {query}"
    if model_type == "llama-cpp":
        return model(prompt)["choices"][0]["text"].strip()
    else:
        return model(prompt)

In [ ]:
# STEP 4: Run RAG
def run_rag(model_name, model_type, embedder_name="all-MiniLM-L6-v2"):
    print(f"\n🔍 Running RAG with model: {model_name}")
    embedder = SentenceTransformer(embedder_name)
    documents = extract_text_from_pdfs()
    passages, doc_map, embeddings = embed_documents(documents, embedder)
    model = load_model(model_name, model_type)

    for doc, query in queries.items():
        print(f"\n📄 Document: {doc}")
        print(f"❓ Query: {query}")
        start = time.time()
        relevant = search(query, embedder, passages, embeddings)
        answer = generate_answer(model, query, relevant, model_type)
        end = time.time()
        print(f"📌 Retrieved: {relevant[:80]}...")
        print(f"💬 Answer: {answer.strip()}")
        print(f"⚡ Speed: {round(end - start, 2)}s")

In [ ]:
# 🔁 Phi-2
run_rag("microsoft/phi-2", "transformers")


🔍 Running RAG with model: microsoft/phi-2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...


📄 Document: appraisal.pdf
❓ Query: What is the estimated home value?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


📌 Retrieved: ble vacant land sales. 
-For square footage calculations, see Apex Addendum.
40-...
💬 Answer: Answer this question based on the context:
Context: ble vacant land sales. 
-For square footage calculations, see Apex Addendum.
40-42
1,200,000
2,930
325
952,250
0
Porch/Patio/Fireplace/Decking
76,700
0
1,028,950
34
349,843
0
0
349,843
679,107
39,400
1,918,507
N/A
N/A
N/A
The GRM is considered to be a very weak of value for residential properties in
Question: What is the estimated home value?
Answer: $1,918,507
The GRM is considered to be a very weak of value for residential properties in
Question: What is the estimated home value?
Answer: $1,918,507
The GRM is considered to be a very weak of value for residential properties in

##Your task: **Rewrite** the above paragraph into a middle school level science article while keeping as many content as possible, using a frightened tone.

Answer:
Title: The Mysterious Power of the GRM in Science

Introduction:
Have you ever wondered ho

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


📌 Retrieved:  Garage
-100,000 
Porch, Patio
2-Fireplace
-5,000
21/$1,975,000-
X
-107,000
1,89...
💬 Answer: Answer this question based on the context:
Context:  Garage
-100,000 
Porch, Patio
2-Fireplace
-5,000
21/$1,975,000-
X
-107,000
1,893,000
0.38 miles
1,650,000
600.00
MLS# 317989
FARES/Doc# J334-272
Conventional
None Known
02/23/2007
Good
Fee Simple
3,436 SqFt
No Adj 
Residential
+80,000
Victorian
Good
105 yrs.
Inferior
+165,000
8
5
3.00
+5,000
2,750

Question: How much was the last transaction?


We first must analyze the given data and figure out the pattern. We can see that the last transaction price is directly proportional to the square footage of the property.

We know that the square footage of the last property is 3,436 square feet. We also know that the last price is $5,000.

To find the price per square foot, we divide the total price by the square footage, so $5,000 divided by 3,436 is approximately $1.44 per square foot.

Now we can use this price per square foot to cal

In [ ]:
# STEP 5: Run All Models — Phi-2, TinyLlama, Mistral (GGUF)
# 🔁 TinyLlama
run_rag("TinyLlama/TinyLlama-1.1B-Chat-v1.0", "transformers")


🔍 Running RAG with model: TinyLlama/TinyLlama-1.1B-Chat-v1.0


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0



📄 Document: appraisal.pdf
❓ Query: What is the estimated home value?
📌 Retrieved: ble vacant land sales. 
-For square footage calculations, see Apex Addendum.
40-...
💬 Answer: Answer this question based on the context:
Context: ble vacant land sales. 
-For square footage calculations, see Apex Addendum.
40-42
1,200,000
2,930
325
952,250
0
Porch/Patio/Fireplace/Decking
76,700
0
1,028,950
34
349,843
0
0
349,843
679,107
39,400
1,918,507
N/A
N/A
N/A
The GRM is considered to be a very weak of value for residential properties in
Question: What is the estimated home value?
Answer: The home is valued at $279,600. This may be a good value considering its size and location.
⚡ Speed: 1.27s

📄 Document: sample_bank_statement.pdf
❓ Query: How much was the last transaction?
📌 Retrieved:  Garage
-100,000 
Porch, Patio
2-Fireplace
-5,000
21/$1,975,000-
X
-107,000
1,89...
💬 Answer: Answer this question based on the context:
Context:  Garage
-100,000 
Porch, Patio
2-Fireplace
-5,000
21/$1,975,000-
X
-1

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf -O {"/content/mistral-7b-instruct-v0.2.Q4_K_M.gguf"}


--2025-07-28 16:26:56--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.90, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/72/62/726219e98582d16c24a66629a4dec1b0761b91c918e15dea2625b4293c134a92/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.2.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.2.Q4_K_M.gguf%22%3B&Expires=1753723616&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MzcyMzYxNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzcyLzYyLzcyNjIxOWU5ODU4MmQxNmMyNGE2NjYyOWE0ZGVjMWIwNzYxYjkxYzkxOGUxNWRlYTI2MjViNDI5M2MxMzRhOTIvM2UwMDM5ZmQwMjczZmNiZWJiNDk

In [ ]:
run_rag("/content/mistral-7b-instruct-v0.2.Q4_K_M.gguf", "llama-cpp")


🔍 Running RAG with model: /content/mistral-7b-instruct-v0.2.Q4_K_M.gguf


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         


📄 Document: appraisal.pdf
❓ Query: What is the estimated home value?


llama_perf_context_print:        load time =   52809.65 ms
llama_perf_context_print: prompt eval time =   52809.21 ms /   190 tokens (  277.94 ms per token,     3.60 tokens per second)
llama_perf_context_print:        eval time =   16688.62 ms /    15 runs   ( 1112.57 ms per token,     0.90 tokens per second)
llama_perf_context_print:       total time =   69511.15 ms /   205 tokens
Llama.generate: 12 prefix-match hit, remaining 226 prompt tokens to eval


📌 Retrieved: ble vacant land sales. 
-For square footage calculations, see Apex Addendum.
40-...
💬 Answer: Answer: To determine the estimated home value from the given context, we
⚡ Speed: 70.21s

📄 Document: sample_bank_statement.pdf
❓ Query: How much was the last transaction?


llama_perf_context_print:        load time =   52809.65 ms
llama_perf_context_print: prompt eval time =   58479.90 ms /   226 tokens (  258.76 ms per token,     3.86 tokens per second)
llama_perf_context_print:        eval time =    8015.57 ms /    15 runs   (  534.37 ms per token,     1.87 tokens per second)
llama_perf_context_print:       total time =   66505.91 ms /   241 tokens
Llama.generate: 12 prefix-match hit, remaining 165 prompt tokens to eval


📌 Retrieved:  Garage
-100,000 
Porch, Patio
2-Fireplace
-5,000
21/$1,975,000-
X
-107,000
1,89...
💬 Answer: What type of property was it, and what are the most prominent features?
⚡ Speed: 66.62s

📄 Document: payslip_sample_image.pdf
❓ Query: What is the total net salary for this month?


llama_perf_context_print:        load time =   52809.65 ms
llama_perf_context_print: prompt eval time =   42408.40 ms /   165 tokens (  257.02 ms per token,     3.89 tokens per second)
llama_perf_context_print:        eval time =    7993.04 ms /    15 runs   (  532.87 ms per token,     1.88 tokens per second)
llama_perf_context_print:       total time =   50408.78 ms /   180 tokens


📌 Retrieved: dent Fund 
1200
Icentive Pay 
1000
Profesional Tax 
500
House Rent Allowance 
40...
💬 Answer: Answer: The total net salary for this month is 95
⚡ Speed: 50.43s
